pandas-ylt
==========

Python modules for working in pandas with various loss data tables used for CAT modelling


### Read the example YLT

In [1]:
import pandas as pd
from pandas_ylt.yeareventloss import YearEventLossTable

yelt = pd.read_csv("tests/_data/example_pareto_poisson_yelt.csv",
                    index_col=['Year', 'EventID', 'DayOfYear'])

yelt = YearEventLossTable(yelt['Loss'], n_yrs=1e5)
display(yelt.to_ep_summaries([200, 100, 10]).style.format("{:,.0f}"))
display(yelt.to_aal_series().to_frame())

CurveType,LossAEP,LossOEP
ReturnPeriod,,
200,"104,018","101,450"
100,"51,611","49,340"
10,"5,539","4,708"


,
AAL,5563.829298
STD,158881.066882


### Define some layers

In [2]:
# Define the layers
from pandas_ylt.layer import Layer


layers = [
    # First loss
    {'name': '20 xs 20', 'layer': Layer(**{'limit': 20000, 'xs': 20000, 'agg_limit': 2000.0})},
    {'name': '20 xs 40', 'layer': Layer(**{'limit': 20000, 'xs': 40000, 'agg_limit': 2000.0})},
    {'name': '20 xs 60', 'layer': Layer(**{'limit': 20000, 'xs': 60000, 'agg_limit': 2000.0})},    
    {'name': '20 xs 80', 'layer': Layer(**{'limit': 20000, 'xs': 80000, 'agg_limit': 2000.0})},    
    # Second loss
    {'name': '20 xs 20 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 20000, 'agg_limit': 2000.0, 'agg_xs': 2000.0})},
    {'name': '20 xs 40 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 40000, 'agg_limit': 2000.0, 'agg_xs': 2000.0})},
    {'name': '20 xs 60 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 60000, 'agg_limit': 2000.0, 'agg_xs': 2000.0})}, 
    {'name': '20 xs 80 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 80000, 'agg_limit': 2000.0, 'agg_xs': 2000.0})}, 
]



### Calculate the layer loss

In [3]:
# lyr_loss = layers[4]['layer'].yelt_loss(yelt)
lyr_loss = layers[0]['layer'].yelt_loss(yelt)
lyr_loss.sort_index()

lyr_loss.loc[[2544, 58731, 99929]]
# lyr_loss.loc[[58731]]
# lyr_loss.loc[lyr_loss.AggLoss > 0].join(tmp, rsuffix='tmp')
# occ_loss.loc[occ_loss > 0]

Year   EventID  DayOfYear
2544   40245    181          2000.000000
       29031    206             0.000000
58731  19326    168           283.758114
       17349    227          1716.241886
99929  2111     314          1459.418871
Name: Loss, dtype: float64